<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_gen_chat_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer,pipeline
from datasets import Dataset,load_dataset
ds = load_dataset("zhengr/alpaca-chinese-dataset",cache_dir="./")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['input', 'en_instruction', 'en_output', 'instruction', 'en_input', 'output'],
        num_rows: 1000
    })
})

In [4]:
a = tokenizer("你好,我是小明")
a

{'input_ids': [11877, 15, 6948, 1200, 1407], 'attention_mask': [1, 1, 1, 1, 1]}

In [5]:
ds["train"][0]

{'input': '',
 'en_instruction': 'How can we reduce water usage in our daily lives?',
 'en_output': '1. Use water-efficient fixtures like showerheads and faucets. \n2. Use a basin or bucket to collect household wastewater tasks such as washing dishes and taking baths. \n3. Raise awareness of water-saving practices in our community. \n4. Check for water leaks in plumbing and irrigation systems and repair them promptly. \n5. Take shorter showers and use low flow showerheads to save water. \n6. Collect rainwater and use it for gardening or other non-drinking purposes. \n7. Turn off the tap when you are brushing your teeth or soaping your hands. \n8. Reduce the time you water your lawn. \n9. Reuse graywater (water from washing machine, bathroom sinks, and showers) as much as possible.\n10. Buy only energy-efficient dishwashers and washing machines.',
 'instruction': '我们如何减少日常生活中的用水量？',
 'en_input': '',
 'output': '1. 使用节水的洁具，如淋浴喷头和水龙头。\n2.使用水盆或桶收集家庭废水，例如洗碗和洗澡。\n3. 提高社区节水意识。\n4. 检查管道和灌溉系统的漏

In [6]:
def data_process(example): #这次数据处理不进行batched，只处理单个的数据，因为label部分不太容易做
  max_length=256
  tokenized_input=tokenizer("\n".join(["User:"+example["instruction"],example["input"]]).strip()+"\nAssistant:")
  tokenized_output=tokenizer(example["output"]+tokenizer.eos_token)
  input_ids=tokenized_input["input_ids"]+tokenized_output["input_ids"]
  attention_mask=tokenized_input["attention_mask"]+tokenized_output["attention_mask"]
  labels= [-100]*len(tokenized_input["input_ids"])+tokenized_output["input_ids"]
  if len(input_ids)>max_length:
    input_ids=input_ids[:max_length]
    attention_mask=attention_mask[:max_length]
    labels=labels[:max_length]
  return {
      "input_ids":input_ids,
      "attention_mask":attention_mask,
      "labels":labels
  }

In [7]:
tokenized_ds=ds.map(data_process,remove_columns=ds["train"].column_names)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [8]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'User:我们如何减少日常生活中的用水量？\nAssistant:1. 使用节水的洁具，如淋浴喷头和水龙头。\n2.使用水盆或桶收集家庭废水，例如洗碗和洗澡。\n3. 提高社区节水意识。\n4. 检查管道和灌溉系统的漏水情况，并及时修复。\n5. 淋浴时间较短，使用低流量淋浴喷头以节约用水。\n6.收集雨水并将其用于园艺或其他非饮用目的。\n7. 刷牙或洗手时关闭水龙头。\n8. 减少给草坪浇水的时间。\n9. 尽可能重复使用灰水（洗衣机、浴室水槽和淋浴的水）。\n10. 只购买节能洗碗机和洗衣机。</s>'

In [9]:
#检查label是否只针对output
tmp=list(filter(lambda x:x != -100,tokenized_ds["train"][0]["labels"]))
tokenizer.decode(tmp)

'1. 使用节水的洁具，如淋浴喷头和水龙头。\n2.使用水盆或桶收集家庭废水，例如洗碗和洗澡。\n3. 提高社区节水意识。\n4. 检查管道和灌溉系统的漏水情况，并及时修复。\n5. 淋浴时间较短，使用低流量淋浴喷头以节约用水。\n6.收集雨水并将其用于园艺或其他非饮用目的。\n7. 刷牙或洗手时关闭水龙头。\n8. 减少给草坪浇水的时间。\n9. 尽可能重复使用灰水（洗衣机、浴室水槽和淋浴的水）。\n10. 只购买节能洗碗机和洗衣机。</s>'

In [10]:
args = TrainingArguments(
    output_dir="./gen_chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=10
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [11]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.393900


KeyboardInterrupt: 

In [16]:
# model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/nlp/gen_chatbot/checkpoint-10")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
input = "User: {}\n{}".format("心情不好的时候可以做什么缓解一下？", "").strip() + "\nAssistant: "
pipe(input, max_length=1028, do_sample=True, )

[{'generated_text': 'User: 心情不好的时候可以做什么缓解一下？\nAssistant: 心情不好的时候可以 做 哪些 事情呢？\n1.放松或消减日常生活中感到紧张或愤怒的各种情绪。例如愤怒、烦躁、恼怒、愤怒等愤怒或愤怒的感觉。'}]